In [29]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [30]:
# importing all the necessary libraries
import numpy as np 
import pandas as pd 
import json 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors 
from sklearn.cluster import KMeans 
from bs4 import BeautifulSoup 
from geopy.geocoders import Nominatim 
import folium 
from pandas.io.json import json_normalize 
print("Libraries imported.")

Libraries imported.


In [31]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [32]:
# parsing data using BeautifulSoup
soup = BeautifulSoup(data, 'html.parser')

In [33]:
# initiating lists that will be used to create dataframe
postalCodeList = []
boroughList = []
neighborhoodList = []

In [34]:
# looking for any tables on the Wikipedia page
soup.find('table').find_all('tr')

soup.find('table').find_all('tr')

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [37]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [38]:
# creating the dataframe with the columns we want using pandas
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [40]:
# cleaning up the column values and taking off the unnessary "\n"
toronto_df['Neighborhood'] = toronto_df['Neighborhood'].str.replace("\n","")
toronto_df['Borough'] = toronto_df['Borough'].str.replace("\n","")
toronto_df['PostalCode'] = toronto_df['PostalCode'].str.replace("\n","")
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [41]:
# ignoring the cells that have Borough =! 'Not Assigned'
toronto_df_drop = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_drop.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [42]:
# grouping the neighborhoods with the same Borough on the same row
toronto_df_grouped = toronto_df_drop.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae"


In [43]:
# making Neighborhood == 'Not Assigned' to same Borough name
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae"


In [44]:
toronto_df_grouped['PostalCode'].unique()

array(['M1B', 'M1C', 'M1E', 'M1G', 'M1H', 'M1J', 'M1K', 'M1L', 'M1M',
       'M1N', 'M1P', 'M1R', 'M1S', 'M1T', 'M1V', 'M1W', 'M1X', 'M2H',
       'M2J', 'M2K', 'M2L', 'M2M', 'M2N', 'M2P', 'M2R', 'M3A', 'M3B',
       'M3C', 'M3H', 'M3J', 'M3K', 'M3L', 'M3M', 'M3N', 'M4A', 'M4B',
       'M4C', 'M4E', 'M4G', 'M4H', 'M4J', 'M4K', 'M4L', 'M4M', 'M4N',
       'M4P', 'M4R', 'M4S', 'M4T', 'M4V', 'M4W', 'M4X', 'M4Y', 'M5A',
       'M5B', 'M5C', 'M5E', 'M5G', 'M5H', 'M5J', 'M5K', 'M5L', 'M5M',
       'M5N', 'M5P', 'M5R', 'M5S', 'M5T', 'M5V', 'M5W', 'M5X', 'M6A',
       'M6B', 'M6C', 'M6E', 'M6G', 'M6H', 'M6J', 'M6K', 'M6L', 'M6M',
       'M6N', 'M6P', 'M6R', 'M6S', 'M7A', 'M7R', 'M7Y', 'M8V', 'M8W',
       'M8X', 'M8Y', 'M8Z', 'M9A', 'M9B', 'M9C', 'M9L', 'M9M', 'M9N',
       'M9P', 'M9R', 'M9V', 'M9W'], dtype=object)

In [45]:
# creating the dataset we need with the given conditions for the assignment
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,"Central Bay Street, Central Bay Street"
1,M2H,North York,"Hillcrest Village, Hillcrest Village"
2,M4B,East York,"Parkview Hill, Woodbine Gardens, Parkview Hill..."
3,M1J,Scarborough,"Scarborough Village, Scarborough Village"
4,M4G,East York,"Leaside, Leaside"
5,M4M,East Toronto,"Studio District, Studio District"
6,M1R,Scarborough,"Wexford, Maryvale, Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,"Humber Summit, Humber Summit"
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


In [46]:
print('The shape of the dataset is: {}'.format(toronto_df_grouped.shape))

The shape of the dataset is: (103, 3)


### Question 2

In [47]:
# using the Geospatial_Coordinates.csv to get respective coordinates for each postal code
df_geo_coor = pd.read_csv("Geospatial_Coordinates.csv")
df_geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [48]:
# merging both datasets together
df_toronto = pd.merge(toronto_df_grouped, df_geo_coor, how='left', left_on = 'PostalCode', right_on = 'Postal Code')

# remove the "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn, Woburn",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae, Cedarbrae",43.773136,-79.239476


In [49]:
# dataframe with necessary columns specified in the question along with the coordinates
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(df_toronto[df_toronto["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,"Central Bay Street, Central Bay Street",43.657952,-79.387383
1,M2H,North York,"Hillcrest Village, Hillcrest Village",43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens, Parkview Hill...",43.706397,-79.309937
3,M1J,Scarborough,"Scarborough Village, Scarborough Village",43.744734,-79.239476
4,M4G,East York,"Leaside, Leaside",43.709060,-79.363452
5,M4M,East Toronto,"Studio District, Studio District",43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale, Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,"Humber Summit, Humber Summit",43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


### Question 3

In [50]:
# getting the latitude and longitude of Toronto
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [51]:
# creating map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

In [52]:
# adding in the markers for Toronto neighborhoods
for lat, lng, borough, neighborhood in zip(
        df_toronto['Latitude'], 
        df_toronto['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [53]:
# making sure to only display boroughs that contain the word Toronto
df_toronto_denc = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_denc.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,"The Beaches, The Beaches",43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale, The Danforth Wes...",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West, India Bazaar, ...",43.668999,-79.315572
3,M4M,East Toronto,"Studio District, Studio District",43.659526,-79.340923
4,M4N,Central Toronto,"Lawrence Park, Lawrence Park",43.728020,-79.388790


In [54]:
# display map for only those boroughs with Toronto in name
map_toronto_denc = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df_toronto_denc['Latitude'], 
        df_toronto_denc['Longitude'], 
        df_toronto_denc['Borough'], 
        df_toronto_denc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_denc)  

map_toronto_denc

In [55]:
# specifying credentials for Foursquare
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = ''

In [56]:
# assigning a value to neighborhood_name with which we'll work with
neighborhood_name = df_toronto_denc.loc[3, 'Neighborhood']
print(f"The first neighborhood's name is '{neighborhood_name}'.")

The first neighborhood's name is 'Studio District, Studio District'.


In [57]:
# obtaining the longitude and latitude of the neighborhood we are using
neighborhood_latitude = df_toronto_denc.loc[3, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto_denc.loc[3, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Studio District, Studio District are 43.6595255, -79.340923.


In [58]:
# just a sanity check to see if our indexing was correct above
df_toronto_denc.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,"The Beaches, The Beaches",43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale, The Danforth Wes...",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West, India Bazaar, ...",43.668999,-79.315572
3,M4M,East Toronto,"Studio District, Studio District",43.659526,-79.340923
4,M4N,Central Toronto,"Lawrence Park, Lawrence Park",43.728020,-79.388790
